In [ ]:
import os
import glob
sacred_dir = "sacred_cluster"
parent_dir = os.getcwd()
print(os.getcwd())
cout_json_files = glob.glob(os.path.join(parent_dir, sacred_dir, "**/*cout.txt"), recursive=True)
test_files = glob.glob(os.path.join(parent_dir, sacred_dir, "test**/*cout.txt"), recursive=True)
cout_json_files = set(cout_json_files)-set(test_files)
print(len(cout_json_files))
print(len(test_files))

In [ ]:
from datetime import datetime, timedelta
import json

dict_infos={}
for i in test_files:
    name = os.path.basename(os.path.dirname(i))
    hours = []
    first_hour=[False, None]
    last_hour=None

    with open(os.path.join(i), "r") as f:
        lines = f.readlines()
        hours.append(lines[0][6:14])
        for line in lines:
            if line.startswith("[INFO") and line[6:14][:2] != hours[-1][:2]:
                hours.append(line[6:14])
            # store first seen and last seen hour
            if first_hour[0] == False:
                first_hour[0] = True
                first_hour[1] = line[6:14]
            last_hour = line[6:14]

        cpt_days = 0
        while len(hours) > 25:
            hours = hours[:1] + hours[25:]
            cpt_days += 1
        print("-----------------------------------------")
        FMT = '%H:%M:%S'
        tdelta = datetime.strptime(last_hour, FMT) - datetime.strptime(first_hour[1], FMT)
        if tdelta.days < 0:
            cpt_days += 1
        total_time = tdelta + timedelta(days=cpt_days)
    # open info file (name/info.json) and get the name of the hardware
    with open(os.path.join(os.path.dirname(i), "run.json"), "r") as f:
        run = json.load(f)
        cpu = run["host"]["cpu"]
        if "gpus" in run["host"]:
            gpu=run["host"]["gpus"]["gpus"][0]["model"]

        else:
            gpu=None
    print(total_time, first_hour[1], last_hour, name, cpu, gpu)
    dict_infos[name] = total_time.total_seconds()/(60*60)
    dict_infos[name]={
        "name": name,
        "total_time": total_time.total_seconds()/(60*60),
        "cpu": cpu,
        "gpu": gpu
    }

In [ ]:
new_dict = {}
for k, v in dict_infos.items():
    # k finishes by "_2023-04-03_02-32-12__2023-04-11_03-47-36", remove it
    # k also starts by "test_10000_", remove it
    k = k[11:-42]

    # k is now "qvmix_uc_50_struct_uc_50_cc_True" or 'facmac_uc_10_struct_uc_5_cc_True", we want "qvmix_struct_uc_50_cc_True" or "facmac_struct_uc_5_cc_True"
    k=k.split("_")
    if "cc" in k:
        if "owf" not in k:
            k = k[0] + "_" + k[4] + "_" + k[5] + "_" + k[6] + "_" + k[7]
        else:
            if "dqn" in k:
                k = k[0] + "_" + k[3] + "_" + k[5] + "_" + k[6] + "_" + k[7]
            else:
                k = k[0] + "_" + k[3] + "_" + k[4] + "_" + k[5] + "_" + k[6]
    else:
        if "owf" not in k:
            k = k[0] + "_" + k[4] + "_" + k[5]
        else:
            if "dqn" in k:
                k = k[0] + "_" + k[3] + "_" + k[5]
            else:
                k = k[0] + "_" + k[3] + "_" + k[4]
    new_dict[k]=new_dict.get(k, [])+[v]

for k, v in new_dict.items():
    print(k, len(v))
    break
print(len(new_dict.keys()))

In [ ]:
methods = ["qmix", "qvmix", "coma", "iql", "qplex", "dqn", "facmac"]
env1 = ["uc", "c", "owf"]
env2 = {"uc": ["3", "5", "10", "50", "100"],
        "c": ["3", "5", "10", "50", "100"], "owf": ["1", "2", "5", "25", "50"]}
ccs = [True, False]
for i in methods:
    for j in env1:
        for k in env2[j]:
            for cc in ccs:
                key = f"{i}_{j}_{k}_cc_{cc}"
                if key in new_dict:
                    pass
                else:
                    key_simple = f"{i}_{j}_{k}"
                    if key_simple in new_dict:
                        pass
                    else:
                        print("miss", key)


In [ ]:
set_cpu = set()
set_gpu=set()
set_pair=set()
corresponding_k={"3":"1", "5":"2", "10":"5", "50":"25", "100":"50"}

for i in methods:
    for j in env1:
        for k in env2[j]:
            for cc in ccs:
                key = f"{i}_{j}_{k}_cc_{cc}"
                if key in new_dict:
                    pass
                else:
                    key_simple = f"{i}_{j}_{k}"
                    if key_simple in new_dict:
                        key = key_simple
                    else:
                        print("error", key)
                        continue
                for valuesss in new_dict[key]:
                    pair=(valuesss["cpu"], valuesss["gpu"])
                    set_cpu.add(valuesss["cpu"])
                    if valuesss["gpu"] is not None:
                        set_gpu.add(valuesss["gpu"])
                        set_pair.add(pair)
print(len(set_cpu))
print(len(set_gpu))
print(len(set_pair))
print(set_pair)


In [ ]:
colors = ['red', 'green', 'blue', 'cyan', 'magenta', 'orange', 'purple', 'brown', 'gray', 'yellow']
print(len(colors))

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

def fig_with_n(k, ax, scatter_handles, scatter_labels):

    methods = ["qmix", "qvmix", "qplex", "coma", "facmac", "iql", "dqn"]
    env1 = ["uc", "c", "owf"]
    env2 = {"uc": ["3", "5", "10", "50", "100"],
            "c": ["3", "5", "10", "50", "100"], "owf": ["1", "2", "5", "25", "50"]}
    ccs = [True, False]
    idx=0
    full=[]
    full_legend=[]
    corresponding_k={"3":"1", "5":"2", "10":"5", "50":"25", "100":"50"}
    set_cpu=set()
    max_found=[None, 0]
    for i in methods:
        for j in env1:
            if j == 'owf':
                real_k = corresponding_k[k]
            else:
                real_k = k
            for cc in ccs:
                key = f"{i}_{j}_{real_k}_cc_{cc}"
                if key in new_dict:
                    pass
                else:
                    key_simple = f"{i}_{j}_{real_k}"
                    if key_simple in new_dict:
                        key = key_simple
                    else:
                        print("error", key)
                        continue

                for data in new_dict[key]:
                    cpu_gpu_pair = data["cpu"]
                    color = color_map[cpu_gpu_pair]
                    scatter=ax.scatter(idx, data["total_time"], color=color, s=500)
                    if cpu_gpu_pair not in scatter_labels:
                        scatter_handles.append(scatter)
                        scatter_labels.append(cpu_gpu_pair)
                list_time = [i["total_time"] for i in new_dict[key]]
                for time__ in list_time:
                    if time__ > max_found[1]:
                        max_found[1] = time__
                        max_found[0] = key
                list_legend=[i["cpu"] for i in new_dict[key]]
                for cpu_name in list_legend:
                    set_cpu.add(cpu_name)
                full+=list_time
                full_legend+=list_legend
            full=[]
            full_legend=[]
            idx+=1
            print(i, j, k)
    fig.canvas.draw()
    ax.xaxis.set_tick_params(length=15, width=5, labelsize=0)
    ax.yaxis.set_tick_params(length=15, width=5, labelsize=50)
    ax.set_ylabel("Time (in hours)", fontsize=50)
    if k == "3" or k == "5":
        ax.set_title(f"{k} or {str(int(corresponding_k[k])*2)} agents", fontsize=50)
    else:
        ax.set_title(f"{k} agents", fontsize=50)
    print("set cpu:", set_cpu)
    print("max_found", max_found, new_dict[max_found[0]])
    return scatter_handles, scatter_labels

fig, axes = plt.subplots(1, 3)
fig.set_figheight(20)
fig.set_figwidth(60)

scatter_handles=[]
scatter_labels=[]

color_map = {pair: color for pair, color in zip(set_cpu, colors)}

scatter_handles, scatter_labels=fig_with_n("3", axes[0], scatter_handles, scatter_labels)
scatter_handles, scatter_labels=fig_with_n("5", axes[1], scatter_handles, scatter_labels)
scatter_handles, scatter_labels=fig_with_n("10", axes[2], scatter_handles, scatter_labels)
fig.suptitle("Experiments with CPU only", fontsize=50)

axbox = axes[1].get_position()
legend = fig.legend(scatter_handles, scatter_labels, bbox_to_anchor=[0.012, axbox.y0-.6365,1,1], loc="center", ncol=2, prop={'size': 50})

fig.canvas.draw()
plt.plot()
plt.savefig(f"results_scripts/plotstesting_time_cpu.pdf",
            bbox_inches='tight')
print(methods, env1)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

def fig_with_n_gpu(k, ax, scatter_handles, scatter_labels):

    methods = ["qmix", "qvmix", "qplex", "coma", "facmac", "iql", "dqn"]
    env1 = ["uc", "c", "owf"]
    env2 = {"uc": ["3", "5", "10", "50", "100"],
            "c": ["3", "5", "10", "50", "100"], "owf": ["1", "2", "5", "25", "50"]}
    ccs = [True, False]
    idx=0
    full=[]
    full_legend=[]
    corresponding_k={"3":"1", "5":"2", "10":"5", "50":"25", "100":"50"}
    set_cpu=set()
    max_found=[None, 0]
    for i in methods:
        for j in env1:
            if j == 'owf':
                real_k = corresponding_k[k]
            else:
                real_k = k
            for cc in ccs:
                key = f"{i}_{j}_{real_k}_cc_{cc}"
                if key in new_dict:
                    pass
                else:
                    key_simple = f"{i}_{j}_{real_k}"
                    if key_simple in new_dict:
                        key = key_simple
                    else:
                        print("error", key)
                        continue
                for data in new_dict[key]:
                    cpu_gpu_pair = (data["cpu"], data["gpu"])
                    color = color_map[cpu_gpu_pair]

                    scatter= ax.scatter(idx, data["total_time"], color=color, s=500)
                    if cpu_gpu_pair not in scatter_labels:
                        scatter_handles.append(scatter)
                        scatter_labels.append(cpu_gpu_pair)
                list_time = [i["total_time"] for i in new_dict[key]]
                for time__ in list_time:
                    if time__ > max_found[1]:
                        max_found[1] = time__
                        max_found[0] = key
                list_legend=[i["cpu"] for i in new_dict[key]]
                for cpu_name in list_legend:
                    set_cpu.add(cpu_name)
                full+=list_time
                full_legend+=list_legend
            # ax.scatter([idx for _ in full],full, s=500, label=full_legend)
            full=[]
            full_legend=[]
            idx+=1
    fig.canvas.draw()
    ax.xaxis.set_tick_params(length=15, width=5, labelsize=0)
    ax.yaxis.set_tick_params(length=15, width=5, labelsize=50)
    ax.set_ylabel("Time (in hours)", fontsize=50)
    if k == "3" or k == "5":
        ax.set_title(f"{k} or {str(int(corresponding_k[k])*2)} agents", fontsize=50)
    else:
        ax.set_title(f"{k} agents", fontsize=50)
    print("set cpu:", set_cpu)
    print("max_found", max_found, new_dict[max_found[0]])
    return scatter_handles, scatter_labels
fig, axes = plt.subplots(1, 2)
fig.set_figheight(20)
fig.set_figwidth(60)

scatter_handles=[]
scatter_labels=[]
color_map = {pair: color for pair, color in zip(set_pair, colors)}
scatter_handles, scatter_labels = fig_with_n_gpu("50", axes[0], scatter_handles, scatter_labels)
scatter_handles, scatter_labels = fig_with_n_gpu("100", axes[1], scatter_handles, scatter_labels)
fig.suptitle("Experiments with CPU and GPU", fontsize=50)

print(scatter_labels)
scatter_labels=[i[0]+" / "+i[1] for i in scatter_labels]
axbox = axes[1].get_position()
legend = fig.legend(scatter_handles, scatter_labels, bbox_to_anchor=[0.012, axbox.y0-.6365,1,1], loc="center", ncol=2, prop={'size': 50})
fig.canvas.draw()
plt.plot()
plt.savefig(f"results_scripts/plots/testing_time_gpu.pdf",
            bbox_inches='tight')
print()